In [1]:
import os

import pandas as pd
pd.options.display.max_colwidth=200

from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

import warnings
warnings.filterwarnings('ignore')

import re
import numpy as np

from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [2]:
train = pd.read_csv('twt_#ifc.csv')
print (train.shape)

(1749, 8)


In [3]:
train.head()


,full_text,retweet_count,user_followers_count,favorite_count,created_at,pos_score,neg_score,sent_score
0,whi wait get fine govern take care wear mask maintain social distanc facemask lockdow,6,2315,0,Mon Jun 01 12:56:02 +0000 2020,0.125,0.375,-1
1,start monday delhi govern gave relax lockdown howev educ institut,1,121,0,Mon Jun 01 12:30:05 +0000 2020,0.125,0.500,-1
2,not meme just inform lockdown look like,0,2,1,Mon Jun 01 12:14:42 +0000 2020,0.000,0.625,-1
3,with lockdown focu reviv economi howev team vijaykarnataka alreadi stimul economi,9,42,0,Mon Jun 01 12:06:29 +0000 2020,0.375,0.500,-1
4,hello guy video get look sonam kapoor celebr inspir look priya inturu lockdown unlock priyainturu fashion trend corona covid,0,3,0,Mon Jun 01 11:54:15 +0000 2020,0.000,0.625,-1


In [4]:
train[train.sent_score==1].head()


,full_text,retweet_count,user_followers_count,favorite_count,created_at,pos_score,neg_score,sent_score
10,upar upar se karlo ba new mean unlock lockdown,7,340,0,Mon Jun 01 11:03:32 +0000 2020,0.375,0.000,1
11,As per order date No state deviat lockdown guidelin unless first advertis awar gener public well advanc how come delhi CM seal delhi border It mockeri order,0,17,0,Mon Jun 01 10:05:36 +0000 2020,1.250,0.500,1
13,keep distanc inde good way stay safe trucker tell long time contact email bhavesh net websit lockdown indiafightscorona logist essentialgood deliveri ship centurycargo,0,7,1,Mon Jun 01 09:58:45 +0000 2020,1.750,0.625,1
14,respons get permiss resum shoot lie broadcast produc ott platform JD majethia http,3,912652,0,Mon Jun 01 09:04:30 +0000 2020,0.375,0.125,1
15,espons get permiss resum shoot lie broadcast produc ott platform JD majethia unlock maharashtragovern lockdown covid,3,85374,14,Mon Jun 01 09:04:17 +0000 2020,0.375,0.125,1


In [5]:
## Postive VS Negative Tweets
print (train.sent_score.value_counts())
print ('% of hate tweets among all tweets',100*2242.0/train.shape[0])

 1    944
-1    576
 0    229
Name: sent_score, dtype: int64
% of hate tweets among all tweets 128.18753573470553


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train["full_text"], train["sent_score"], test_size=0.3, random_state=4242)

train_set = pd.concat([x_train,y_train],axis=1)
test_set = pd.concat([x_test,y_test],axis=1)
train_set.columns = ["full_text","sent_score"]
test_set.columns = ["full_text","sent_score"]
print (train_set.shape)
print (test_set.shape)

(1224, 2)
(525, 2)


In [7]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

remove_words = ['user',"frm", "u", "urs", "n", "ur", "b", "mro", "mo", "tmr", "k", "ok",
"lol", "haha", "w", "moro", "yah", "cya", "cu", "eh", "hm", "hmm",
"yall", "xoxo", "yolo", "em", "v", "ver", "hav", "vry", "shud", "wer",
"abt", "bc", "wen", "jus", "tht", "fr", "hs", "r", "wud", "cud"]

def PreProcess(tweet): 
    #tweet = str(tweet)
    tweet = tweet.lower() ## Convert the text to lower case
    tweet = BeautifulSoup(tweet, "lxml").get_text() ## extract only text by dropping URLS
    tweet=re.sub('[^0-9a-zA-Z]', ' ', tweet) ## extract only alpha numeric from a text
    #tweet = re.sub("[^a-zA-Z]", " ", tweet)
    tweet = ' '.join([word.strip() for word in tweet.split() if word not in stops]) ## Remove eng stop words
    tweet = ' '.join([word for word in tweet.split() if word not in remove_words]) ## Remove specific words
    return tweet

In [8]:
train_set['cleaned_tweet'] = train_set.full_text.apply(PreProcess)
test_set['cleaned_tweet'] = test_set.full_text.apply(PreProcess)

In [9]:
train_set.head()


,full_text,sent_score,cleaned_tweet
202,sanitis carri rajpur road branch dehradun zone covid indiafightscorona http,1,sanitis carri rajpur road branch dehradun zone covid indiafightscorona http
1033,balangir district report fresh covid case,1,balangir district report fresh covid case
336,special song dedic mother sona mohapatra indiafightscorona,0,special song dedic mother sona mohapatra indiafightscorona
1242,indiafightscorona regularli clean commonli touch surfac around stay protect covid healthforal,-1,indiafightscorona regularli clean commonli touch surfac around stay protect covid healthforal
461,I put past moral bankrupt cabinet seed unrest minneapoli tonight you prove everi day lie bed dictat care citizen dead It minneapolisriot covid,1,put past moral bankrupt cabinet seed unrest minneapoli tonight prove everi day lie bed dictat care citizen dead minneapolisriot covid


In [13]:
test_set.head()

,full_text,sent_score,cleaned_tweet
627,busi transform rapidli At gip prepar new way work We never give At gip call businessposit tatasteelspecialeconomiczon gopalpurindustrialpark covid staysaf makeinindia indiafightscorona worldfights...,-1,busi transform rapidli gip prepar new way work never give gip call businessposit tatasteelspecialeconomiczon gopalpurindustrialpark covid staysaf makeinindia indiafightscorona worldfightscorona
423,indiafightscorona Do hesit wait report immedi earli diagnosi share inform hel,-1,indiafightscorona hesit wait report immedi earli diagnosi share inform hel
1496,durham polic say trip barnard castl breach regul health protect coronaviru restrict,1,durham polic say trip barnard castl breach regul health protect coronaviru restrict
1010,A week ago ask member state use covid introduc polici solv controversi issu countri thi includ illeg immigr protection etc some member state never listen,-1,week ago ask member state use covid introduc polici solv controversi issu countri thi includ illeg immigr protection etc member state never listen
1648,vandebharatmiss M V nicobar ship begin journey chennai port blair strand passeng board,0,vandebharatmiss nicobar ship begin journey chennai port blair strand passeng board


In [9]:
### Building TFIDF features### 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2),analyzer='word')


train_tfidf = vectorizer.fit_transform(train_set.cleaned_tweet)

test_tfidf = vectorizer.transform(test_set.cleaned_tweet)

In [11]:
#vectorizer.get_feature_names()
#help(TfidfVectorizer)


In [10]:

#train_tfidf = train_tfidf[4::]
print (train_tfidf.shape)
print (test_tfidf.shape)

(1224, 14840)
(525, 14840)


In [13]:
from sklearn.decomposition import TruncatedSVD
n_comp = 10
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(train_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]

In [11]:

from sklearn.svm import SVC, LinearSVC, NuSVC

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

clf = LinearSVC(C=4)
clf.fit(train_tfidf,train_set.sent_score)
svm_pred_test_label=clf.predict(test_tfidf)

print (accuracy_score(y_true=test_set.sent_score, y_pred=svm_pred_test_label))
print (f1_score(y_true=test_set.sent_score, y_pred=svm_pred_test_label, average='macro'))

0.6666666666666666
0.5947123556016478


In [12]:

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.009)
clf.fit(train_tfidf,train_set.sent_score)
svm_pred_test_label=clf.predict(test_tfidf)

print (accuracy_score(y_true=test_set.sent_score, y_pred=svm_pred_test_label))
print (f1_score(y_true=test_set.sent_score, y_pred=svm_pred_test_label, average='macro'))

 

0.64
0.5486027236027237
